In [114]:
!pip install -U langchain langchain_core langchain_community langchain-text-splitters
!pip install pypdf
!pip install langchain_google_genai
!pip install bs4


In [115]:
!pip install langchain_chroma

In [116]:
# gemini setup
import getpass
import os
os.environ["GOOGLE_API_KEY"]=getpass.getpass("Enter the api key ")

Enter the api key ··········


In [117]:
import bs4
from langchain_core.prompts import PromptTemplate
from langchain_classic.memory import ConversationBufferMemory
from langchain_classic.chains import ConversationChain
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.embeddings import HuggingFaceEmbeddings

In [118]:
llm=ChatGoogleGenerativeAI(model="gemini-2.5-flash",temperature=0)


In [119]:
loder=WebBaseLoader(web_path="https://www.hindustantimes.com/cricket/pakistan-boycott-t20-world-cup-match-against-india-confirm-tournament-participation-101769945127505.html",
                    bs_kwargs=dict(
                        # parse_only=bs4.SoupStrainer(id="main-content")
                        parse_only=bs4.SoupStrainer("div", class_="articlePage")

                    ))



In [120]:
docs=loder.load()

In [121]:
# Re-run the splitting logic with the loaded docs
text_spliter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split = text_spliter.split_documents(docs)

print(f"Number of chunks: {len(split)}")

Number of chunks: 6


In [122]:
text_spliter=RecursiveCharacterTextSplitter(chunk_size=1000,
                                            chunk_overlap=200)
split=text_spliter.split_documents(docs)


In [123]:
split[:3]

[Document(metadata={'source': 'https://www.hindustantimes.com/cricket/pakistan-boycott-t20-world-cup-match-against-india-confirm-tournament-participation-101769945127505.html'}, page_content="Pakistan announces boycott of T20 World Cup match against India; confirms tournament participation, travel to Sri LankaPakistan confirm participation in the T20 World Cup, but the team won't take the field against India on February 15 in Colombo.\xa0Updated on: Feb 01, 2026 11:48 PM ISTWritten by Vishesh RoyShare viaCopy link Pakistan won't pull out of the upcoming T20 World Cup set to be played in India and Sri Lanka from February 7 to March 8, but the team won't be taking the field against arch-rival India on February 15 in Colombo. The decision was confirmed by the Pakistan government on Sunday evening, following a meeting between the country's Prime Minister Shehbaz Sharif and PCB chief Mohsin Naqvi. Ever since the International Cricket Council (ICC) replaced Bangladesh with Scotland in the 20

In [124]:
model_name="BAAI/bge-Base-en-v1.5"
model=HuggingFaceEmbeddings(model_name=model_name,
                            model_kwargs={"device":"cpu"},
                            encode_kwargs={"normalize_embeddings":True})

In [125]:
db=Chroma.from_documents(documents=split,embedding=model)

In [126]:
retriver=db.as_retriever()

In [127]:
from langchain_classic.memory import ConversationBufferMemory

# Initialize memory. Removed return_messages=True to ensure history is passed as a string to the prompt template.
memory = ConversationBufferMemory(memory_key="chat_history")

In [149]:
prompt = PromptTemplate.from_template("""Answer  the question based only on the provided context answer in short and important point only no rubish no extra context:

<context>
{context}
</context>

Question: {input}""")

In [150]:
def format_docs(docs):
  return "\n\n".join(doc.page_content for doc in docs)

In [146]:
chain=({"context":retriver |format_docs ,"input": RunnablePassthrough()} # simply takes the input it receives and passes it along without changing it. It's useful for when you want to include the original input in a chain of operations
       | prompt
       | llm
       | StrOutputParser())

In [151]:
print("🤖 Chatbot ready. Type 'exit' to stop.\n")

while True:
    user_input = input("user: ")
    if user_input.lower() in ["exit", "quit", "end"]:
        print("bye.. 👋")
        break

    answer = chain.invoke( query )

    print("\nChatbot:", answer, "\n")

🤖 Chatbot ready. Type 'exit' to stop.

user: The Government of the Islamic Republic of Pakistan grants approval?

Chatbot: Hey there!

Pakistan announced they won't play their T20 World Cup match against **India**. They're still going to play in the tournament, just not that specific game against India. 

user: quit
bye.. 👋
